# Sparsemax Regression Implementation

We will attempt to replicate the procedures described in section 4.2 of "From Softmax to Sparsemax" (Martins, Astudillo) using the dataset birds.arff. These files can be found at: http://mulan.sourceforge.net/datasets-mlc.html

In particular, we will try to classify a bird as one of 19 mutually non-exlcusive categories. This will be done using three different loss functions: sparsemax, softmax and logistic (sigmoid). In all cases, the optimization will be performed using the  L-BFGS algorithm


## Projecting onto simplex

First, we will define the sparsemax evaluation function, which takes vectors of real numbers and computes a probability vector by projecting onto the K-1 dimensional simplex. 

In [1]:

"""
This function takes a vector of real numbers and projects onto the K-1 dimensional simplex. Implemntation/logic was doublechecked against
an alternative implementation found at: https://github.com/satyammittal/SMAI-Project-Sparsemax/blob/master/code/train_multilabel_classifier.py
"""
def sparsemax_eval(a, radius=1.0):
    x0 = a.copy()
    d = len(x0);
    ind_sort = np.argsort(-x0) # compute order statistics
    y0 = x0[ind_sort] #sort
    ycum = np.cumsum(y0) 
    val = (ycum - radius)/np.arange(1,d+1) # compute critical values
    ind = np.nonzero(y0 > val)[0]
    
    # This edecase deals with the case where large floating point numbers cause (k - (k-1)) to evaluate to the same number. 
    # In this case, we set the largest component to 1 and the remaining components to 0. 
    if len(ind) == 0:
        rho = 0
        tau = val[rho]
        y = y0 - tau
        ind = np.nonzero(y < 0)
        y[ind] = 0
        x = x0.copy()
        x[ind_sort] = y
        x[ind_sort[0]] = 1
    else: 
        rho = ind[-1]
        tau = val[rho]
        y = y0 - tau
        ind = np.nonzero(y < 0)
        y[ind] = 0
        x = x0.copy()
        x[ind_sort] = y

    return x, tau


def softmax_eval(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

## Loading dataset

We will now load in the dataset birds.arff

The file is split into a seperate training and test dataset. The dataset contains 19 columns correspondign to the response vector and 260 potential features to use

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import time,random
import sys,os
import pdb
from scipy.io import arff
import pandas as pd
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics
from past.builtins import xrange
warnings.filterwarnings(action='ignore')

data = arff.loadarff('birds-train.arff')
train = pd.DataFrame(data[0])

data = arff.loadarff('birds-test.arff')
test = pd.DataFrame(data[0])

for col in test.columns: 
    print(col) 

test

audio-ssd1
audio-ssd2
audio-ssd3
audio-ssd4
audio-ssd5
audio-ssd6
audio-ssd7
audio-ssd8
audio-ssd9
audio-ssd10
audio-ssd11
audio-ssd12
audio-ssd13
audio-ssd14
audio-ssd15
audio-ssd16
audio-ssd17
audio-ssd18
audio-ssd19
audio-ssd20
audio-ssd21
audio-ssd22
audio-ssd25
audio-ssd26
audio-ssd27
audio-ssd28
audio-ssd29
audio-ssd30
audio-ssd31
audio-ssd32
audio-ssd33
audio-ssd34
audio-ssd35
audio-ssd36
audio-ssd37
audio-ssd38
audio-ssd39
audio-ssd40
audio-ssd41
audio-ssd42
audio-ssd43
audio-ssd44
audio-ssd45
audio-ssd46
audio-ssd49
audio-ssd50
audio-ssd51
audio-ssd52
audio-ssd53
audio-ssd54
audio-ssd55
audio-ssd56
audio-ssd57
audio-ssd58
audio-ssd59
audio-ssd60
audio-ssd61
audio-ssd62
audio-ssd63
audio-ssd64
audio-ssd65
audio-ssd66
audio-ssd67
audio-ssd68
audio-ssd69
audio-ssd70
audio-ssd73
audio-ssd74
audio-ssd75
audio-ssd76
audio-ssd77
audio-ssd78
audio-ssd79
audio-ssd80
audio-ssd81
audio-ssd82
audio-ssd83
audio-ssd84
audio-ssd85
audio-ssd86
audio-ssd87
audio-ssd88
audio-ssd89
audio-ssd90
a

,audio-ssd1,audio-ssd2,audio-ssd3,audio-ssd4,audio-ssd5,audio-ssd6,audio-ssd7,audio-ssd8,audio-ssd9,audio-ssd10,...,Hermit Warbler,Swainson\'s Thrush,Hammond\'s Flycatcher,Western Tanager,Black-headed Grosbeak,Golden Crowned Kinglet,Warbling Vireo,MacGillivray\'s Warbler,Stellar\'s Jay,Common Nighthawk
0,0.132445,0.143931,0.227729,0.298556,0.385907,0.378363,0.354708,0.384165,0.360092,0.347465,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
1,0.101617,0.130342,0.228117,0.281017,0.365804,0.370122,0.359235,0.388608,0.362013,0.348229,...,b'0',b'0',b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,0.005148,0.017877,0.042137,0.062124,0.097340,0.088305,0.084337,0.083204,0.074532,0.071497,...,b'0',b'1',b'1',b'1',b'0',b'0',b'0',b'0',b'0',b'0'
3,0.018792,0.012898,0.027330,0.039521,0.064671,0.068329,0.065799,0.059891,0.048287,0.047820,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
4,0.007008,0.014610,0.033637,0.042604,0.065649,0.065047,0.064553,0.058155,0.048516,0.047021,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.065968,0.005699,0.009809,0.014150,0.027981,0.027554,0.028538,0.031886,0.027338,0.030704,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
319,0.037432,0.010440,0.021009,0.025018,0.089126,0.037404,0.037024,0.046730,0.033445,0.036546,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
320,0.200058,0.054787,0.137048,0.162441,0.192939,0.177832,0.178606,0.174532,0.136221,0.142075,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
321,0.064331,0.012261,0.022449,0.026526,0.044141,0.040997,0.039509,0.044909,0.040044,0.041884,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'


## Basic data manipulations

We will perform some basic cleaning operations to remain consistent with the paper's procedure, including:
- Standardize predictor variables to zero mean and unit variance
- Remove rows with all 0s as the response

In [109]:

#function to standardize variables to zero mean and unit variance
def normalize(xc):
    ds = xc.copy()
    
    #calculate mean and variance
    ds_mean = ds.mean(axis=0)
    ds_std = ds.std(axis=0)

    #apply transformation 
    for i, features in enumerate(ds):
        ds[i] = features - ds_mean
        for j in range(len(features)):
            if ds_std[j] != 0:
                ds[i][j] / ds_std[j]
    return ds

#for illustration's sake, we will only tain model on the first 50 features included in the dataset
x=train.iloc[:,0:75].to_numpy()
y=train.iloc[:,260:279].to_numpy().astype(int)
weights=np.ones(x.shape[1]*y.shape[1])

xnew=test.iloc[:,0:75].to_numpy()
ynew=test.iloc[:,260:279].to_numpy().astype(int)


x= normalize(x)
xnew = normalize(xnew)

ind_save = np.nonzero(np.sum(y,axis=1) > 0)
ind_save_new = np.nonzero(np.sum(ynew,axis=1) > 0)

x = x[ind_save]
y= y[ind_save]
xnew = xnew[ind_save_new]
ynew = ynew[ind_save_new]



## Defining functions for computing loss and gradient

In [110]:
"""
Computes the support (i.e. non-zero) units of a vector. Returns a vector of 0/1s with 1's corresponding to S(z)
"""
def compute_support(probs):
    ind = probs.nonzero()[0]
    supp =  np.zeros_like(probs)
    supp[ind] = 1.
    return supp

"""
Computes loss function as well as the gradient associated with this loss function

Takes in input of:
- Weight vector corresponding to coefficients for the regression
- X array corresponding to covariate matrix
- Y array corresponding to the response matrix
- Loss function specifying wheter to evaluate using logistic, softmax, or sparsemax loss

Returns output of:
- Loss function
- Vector corresponding to gradient of loss function

"""
def compute_loss(w,x,y,loss_fun):
    
    num = len(x)
    range(num)

    loss = 0

    weights = w.reshape(x.shape[1], y.shape[1])
    loss_gradient  = np.zeros_like(weights)
    
    scores = x.dot(weights)

    for i in range(num):
        s = scores[i,:].copy()
        xeval = x[i,:].copy()
        yeval = y[i,:].copy()
        
        if loss_fun == 'sparsemax':
            z, tau = sparsemax_eval(s)
            support = compute_support(z)
            loss += -s.dot(yeval) +  0.5 * (s**2 - tau**2).dot(support) + 0.5* np.sum(yeval**2)
            delta = -yeval + z
            loss_gradient = np.add(loss_gradient,np.outer(xeval, delta))
        elif loss_fun == 'logistic':
            pr = np.exp(s)  / (1 + np.exp(s))
            log_pr = np.log(pr)
            log_neg_pr = np.log(1-pr)
            loss += -log_pr.dot(yeval)  -(1-yeval).dot(log_neg_pr)
            delta = pr - yeval
            loss_gradient = np.add(loss_gradient,np.outer(xeval, delta))
        elif loss_fun == 'softmax':
            pr = np.exp(s) / np.sum(np.exp(s))
            loss += -s.dot(yeval) + np.log(np.sum(np.exp(s)))
            delta = pr - yeval 
            loss_gradient = np.add(loss_gradient,np.outer(xeval, delta))
    
    loss /= num
    loss_gradient /= num
        
    return loss, loss_gradient.flatten()


"""
Takes in coefficients + probability threshold and returns micro-F1 and macro-F1 metrics based on resulting prediction matrix
assuming a logistic activation functions

Inputs are:
- Weight vector of coefficients
- X array corresponding to covariate matrix
- Y array corresponding to the response matrix
- Probability threshold

Outputs are:
- Matrix of probability outputs
- Matrix of prediction outputs
- Micro-averaged F1 score
- Macro-averaged F1 score

"""

def score_logistic(w,xt,yt,threshold):
    w_score = w.copy()
    w_score = w_score.reshape(x.shape[1], y.shape[1])
    scores = xt.dot(w_score)
    pmatrix = np.exp(scores)/(1+np.exp(scores))
    pred = 1*(pmatrix > threshold)

    micro_f1 = metrics.f1_score(yt,pred,average='micro')
    macro_f1 = metrics.f1_score(yt,pred,average='macro')
    return pred, pmatrix, [micro_f1, macro_f1]

"""
Takes in coefficients + probability threshold and returns micro-F1 and macro-F1 metrics based on resulting prediction matrix
assuming a softmax function

Inputs are:
- Weight vector of coefficients
- X array corresponding to covariate matrix
- Y array corresponding to the response matrix
- Probability threshold

Outputs are:
- Matrix of probability outputs
- Matrix of prediction outputs
- Micro-averaged F1 score
- Macro-averaged F1 score

"""

def score_softmax(w,xt,yt,threshold):
    w_score = w.copy()
    w_score = w_score.reshape(x.shape[1], y.shape[1])
    scores = xt.dot(w_score)
    sc = np.exp(scores)
    sums = np.sum(sc,axis=1)
    pmatrix = sc/sums[:,None]
    pred = 1*(pmatrix > threshold)

    micro_f1 = metrics.f1_score(yt,pred,average='micro')
    macro_f1 = metrics.f1_score(yt,pred,average='macro')
    return pred, pmatrix, [micro_f1, macro_f1]

"""
Takes in coefficients + probability threshold and returns micro-F1 and macro-F1 metrics based on resulting prediction matrix
assuming a sparsemax function

Inputs are:
- Weight vector of coefficients
- X array corresponding to covariate matrix
- Y array corresponding to the response matrix
- Probability threshold

Outputs are:
- Matrix of probability outputs
- Matrix of prediction outputs
- Micro-averaged F1 score
- Macro-averaged F1 score

"""

def score_sparsemax(w,xt,yt,t):
    w_score = w.copy()
    w_score = w_score.reshape(x.shape[1], y.shape[1])
    scores = t*xt.dot(w_score)
    pmatrix = scores.copy()
    
    for i in range(x.shape[1]):
        pmatrix[i] = sparsemax_eval(scores[i])[1]

    pred = 1*(pmatrix > 0)

    micro_f1 = metrics.f1_score(yt,pred,average='micro')
    macro_f1 = metrics.f1_score(yt,pred,average='macro')
    return pred, pmatrix, [micro_f1, macro_f1]






## Hyper-parameter tuning

Tunes probability thresholds for logistic and softmax loss using 5-fols cross validation on the training dataset. 

Based on the results, I use a threshold of 0.65 for logistic regression. I use a threshold of K = 1/19 as a threshold for softmax regression. 

In [111]:

"""
Performs K-folds cross validation

Inputs are:
- Weight vector of coefficients
- X array corresponding to covariate matrix
- Y array corresponding to the response matrix
- Probability threshold

Outputs are:
- Matrix of probability outputs
- Matrix of prediction outputs
- Micro-averaged F1 score
- Macro-averaged F1 score

"""
def crossval(weights, x,y,num_splits,loss_fun):
    kf = KFold(n_splits=num_splits)
    kf.get_n_splits(x)
    num_cats = y.shape[1]
    

    i=0
    metric_list = []
    for train_index, test_index in kf.split(x):
        i= i + 1
        t = train_index
        s = test_index

        x_train= x[t]
        y_train = y[t]
        x_test= x[s]
        y_test = y[s]

        if loss_fun == 'logistic':
            w_k, value, d = opt.fmin_l_bfgs_b(compute_loss,
                                x0=weights,
                                args=(x_train, y_train, 'logistic'),
                                m=10,
                                factr=100,
                                pgtol=1e-08,
                                epsilon=1e-12,
                                approx_grad=False,
                                disp=True,
                                maxfun=1000,
                                maxiter=100)
            for j in range(20): 
                a,b,metric = score_logistic(w_k,x_test,y_test,j/20)
                metric = np.append([i, j/20], metric)
                metric_list.append(metric)
        elif loss_fun == 'softmax':
            w_k, value, d = opt.fmin_l_bfgs_b(compute_loss,
                                x0=weights,
                                args=(x_train, y_train, 'softmax'),
                                m=10,
                                factr=100,
                                pgtol=1e-08,
                                epsilon=1e-12,
                                approx_grad=False,
                                disp=True,
                                maxfun=1000,
                                maxiter=100)
            for j in range(num_cats): 
                a,b,metric = score_softmax(w_k,x_test,y_test,j/num_cats)
                metric = np.append([i, j/num_cats], metric)
                metric_list.append(metric)
    df = pd.DataFrame(metric_list)
    df.columns = ['K','Threshold','F1 Micro-Average','F1 Macro-Average']
    return df.groupby('Threshold').mean()

print(crossval(weights,x,y,5,'logistic'))
print(crossval(weights,x,y,5,'softmax'))


             K  F1 Micro-Average  F1 Macro-Average
Threshold                                         
0.00       3.0          0.182184          0.171570
0.05       3.0          0.194270          0.180325
0.10       3.0          0.199350          0.183129
0.15       3.0          0.206812          0.188219
0.20       3.0          0.210007          0.188839
0.25       3.0          0.212609          0.187569
0.30       3.0          0.217861          0.191187
0.35       3.0          0.216693          0.188962
0.40       3.0          0.225281          0.193475
0.45       3.0          0.223048          0.189602
0.50       3.0          0.225135          0.187934
0.55       3.0          0.227616          0.187208
0.60       3.0          0.229548          0.187147
0.65       3.0          0.231250          0.185564
0.70       3.0          0.227753          0.183963
0.75       3.0          0.219618          0.174602
0.80       3.0          0.223364          0.169409
0.85       3.0          0.22198

## Results for Logistic Regression

In [118]:

w_logistic, value, d = \
      opt.fmin_l_bfgs_b(compute_loss,
                        x0=weights,
                        args=(x, y, 'logistic'),
                        m=10,
                        factr=100,
                        pgtol=1e-08,
                        epsilon=1e-12,
                        approx_grad=False,
                        disp=True,
                        maxfun=1000,
                        maxiter=100)


_,_,logistic_metric = score_logistic(w_logistic,xnew,ynew,0.65)

print("Micro F1 Score: " + str(logistic_metric[0]))
print("Macro F1 Score: " + str(logistic_metric[1]))


Micro F1 Score: 0.21366024518388796
Macro F1 Score: 0.17908162169430974


## Results for Softmax Regression

In [113]:
w_softmax, value, d2 = \
      opt.fmin_l_bfgs_b(compute_loss,
                        x0=weights,
                        args=(x, y, 'softmax'),
                        m=10,
                        factr=100,
                        pgtol=1e-08,
                        epsilon=1e-12,
                        approx_grad=False,
                        disp=True,
                        maxfun=1000,
                        maxiter=10000)


_,_,softmax_metric = score_softmax(w_softmax,xnew,ynew,1/19)

print("Micro F1 Score: " + str(softmax_metric[0]))
print("Macro F1 Score: " + str(softmax_metric[1]))

Micro F1 Score: 0.3001686340640809
Macro F1 Score: 0.17462059849372655


## Results for Sparsemax Regression

In [119]:
    w_sparsemax, value, d = \
      opt.fmin_l_bfgs_b(compute_loss,
                        x0=weights,
                        args=(x, y, 'sparsemax'),
                        m=10,
                        factr=100,
                        pgtol=1e-08,
                        epsilon=1e-12,
                        approx_grad=False,
                        disp=True,
                        maxfun=1000,
                        maxiter=100)
_,_,sparsemax_metric = score_sparsemax(w_sparsemax,x,y,1)

print("Micro F1 Score: " + str(sparsemax_metric[0]))
print("Macro F1 Score: " + str(sparsemax_metric[1]))

Micro F1 Score: 0.21712647796488713
Macro F1 Score: 0.2018138672977463
